In [1]:
from tf_models import ResNet

2023-10-08 06:12:03.869882: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import importlib

import tf_models  # First, import the library
importlib.reload(tf_models)  # Now, reload it


<module 'tf_models' from '/home/zpeng/naoshima/ml/objcla/tf_models.py'>

In [3]:
import click

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.losses import CategoricalCrossentropy
import matplotlib.pyplot as plt
import numpy as np
import dataloader

from tf_models import *


dataset = 'cifar10'


loader = getattr(dataloader, f"load_{dataset}")
x_train, y_train, x_test, y_test = loader(onehot=True)

if x_train[0].ndim == 2:
    x_train = x_train[..., np.newaxis]
    x_test = x_test[..., np.newaxis]
image_shape = x_train[0].shape  # (28, 28)

model = ResNet(image_shape, num_classes=y_train.shape[1], l2_lambda=4e-3)

model.compile(
    optimizer="adam", loss=CategoricalCrossentropy(), metrics=["accuracy"]
)


2023-10-08 06:12:07.571 | INFO     | dataloader:_load_keras:30 - Using cifar10 dataset
2023-10-08 06:12:07.572 | INFO     | dataloader:_load_keras:33 - The size is 50000
2023-10-08 06:12:07.572 | INFO     | dataloader:_load_keras:34 - The shape is: (32, 32, 3)
2023-10-08 06:12:08.325463: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-08 06:12:08.336693: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-08 06:12:08.339505: I tensorflow/compiler/xla/strea

In [ ]:

for i in range(10):
    with tf.device("/GPU:0"):
        history = model.fit(
            x_train, y_train, epochs=4, batch_size=64, validation_split=0.2
        )
        model.save(f'{dataset}-viz.keras')

    # Evaluate the model on the test set
    loss, accuracy = model.evaluate(x_test, y_test)
    print(f"Test loss: {loss:.4f}")
    print(f"Test accuracy: {accuracy:.4f}")

Epoch 1/4


2023-10-08 06:12:14.996860: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-10-08 06:12:16.420127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8900
2023-10-08 06:12:17.146334: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-10-08 06:12:17.694903: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f19ab1e6a90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-08 06:12:17.694941: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-10-08 06:12:17.701359: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_C

625/625 [==============================] - 59s 72ms/step - loss: 5.7952 - accuracy: 0.4719 - val_loss: 3.5800 - val_accuracy: 0.2149
Epoch 2/4
625/625 [==============================] - 44s 70ms/step - loss: 1.8704 - accuracy: 0.5708 - val_loss: 2.8440 - val_accuracy: 0.3077
Epoch 3/4
625/625 [==============================] - 44s 70ms/step - loss: 1.6614 - accuracy: 0.5973 - val_loss: 2.2713 - val_accuracy: 0.4327
Epoch 4/4
313/313 [==============================] - 3s 9ms/step - loss: 2.1390 - accuracy: 0.4682
Test loss: 2.1390
Test accuracy: 0.4682
Epoch 1/4
625/625 [==============================] - 44s 71ms/step - loss: 1.5124 - accuracy: 0.6363 - val_loss: 1.7442 - val_accuracy: 0.5307
Epoch 2/4
625/625 [==============================] - 44s 70ms/step - loss: 1.4425 - accuracy: 0.6542 - val_loss: 2.0594 - val_accuracy: 0.4927
Epoch 3/4
625/625 [==============================] - 44s 70ms/step - loss: 1.4001 - accuracy: 0.6663 - val_loss: 1.6841 - val_accuracy: 0.5572
Epoch 4/4
313

In [ ]:
model.save(f'{dataset}-viz.keras')